In [25]:
import pandas as pd
from IPython.display import display, HTML

In [26]:
dict_cities = {}
cities_file = open("corona_cities_apr_3.txt","r",encoding="utf-8")
index_ = 0
for line_ in cities_file:
    parts = line_.split()
    temp_ints = []
    temp_name = []
    for part_ in parts:
        if part_.isnumeric():
            temp_ints.append(part_)
        else:
            temp_name.append(part_)

    dict_cities[index_] = {'population': int(temp_ints[0]), 'sick': int(temp_ints[1]), 'city names': ' '.join(temp_name)}
    index_ = index_ + 1
print(dict_cities)

{0: {'population': 914559, 'sick': 1132, 'city names': 'ירושלים'}, 1: {'population': 195298, 'sick': 1061, 'city names': 'בני ברק'}, 2: {'population': 450192, 'sick': 337, 'city names': 'תל אביב - יפו'}, 3: {'population': 139032, 'sick': 170, 'city names': 'אשקלון'}, 4: {'population': 242478, 'sick': 138, 'city names': 'פתח תקווה'}, 5: {'population': 241010, 'sick': 132, 'city names': 'ראשון לציון'}, 6: {'population': 213971, 'sick': 126, 'city names': 'נתניה'}, 7: {'population': 46760, 'sick': 120, 'city names': 'אלעד'}, 8: {'population': 120812, 'sick': 118, 'city names': 'בית שמש'}, 9: {'population': 196755, 'sick': 112, 'city names': 'באר שבע'}, 10: {'population': 225073, 'sick': 111, 'city names': 'אשדוד'}, 11: {'population': 143370, 'sick': 106, 'city names': 'רמת גן'}, 12: {'population': 185741, 'sick': 104, 'city names': 'חולון'}, 13: {'population': 130523, 'sick': 102, 'city names': 'בת ים'}, 14: {'population': 272584, 'sick': 96, 'city names': 'חיפה'}, 15: {'population': 7380

In [27]:
cities_pretty = pd.DataFrame.from_dict(dict_cities, orient='index')
cities_pretty['wiki article'] = ["""https://nominatim.openstreetmap.org/search.php?q="""+str(i)+"""&format=json""" for i in cities_pretty['city names']]
cities_pretty['wiki'] = cities_pretty['wiki article'].apply(lambda x: '<a href="{0}">link</a>'.format(x))
display(HTML(cities_pretty.sort_values(by=['population'],ascending=False).head().to_html(escape=False)))

,population,sick,city names,wiki article,wiki
0,914559,1132,ירושלים,https://nominatim.openstreetmap.org/search.php?q=ירושלים&format=json,link
2,450192,337,תל אביב - יפו,https://nominatim.openstreetmap.org/search.php?q=תל אביב - יפו&format=json,link
14,272584,96,חיפה,https://nominatim.openstreetmap.org/search.php?q=חיפה&format=json,link
4,242478,138,פתח תקווה,https://nominatim.openstreetmap.org/search.php?q=פתח תקווה&format=json,link
5,241010,132,ראשון לציון,https://nominatim.openstreetmap.org/search.php?q=ראשון לציון&format=json,link


In [28]:
import requests
import json
import time

def get_geo(link):
    time.sleep(0.5)
    try:
        r = requests.get(link, timeout=1)
        if r.status_code  == 200:
            try:
                data = json.loads(r.content)
                if len(data[0]) > 1:
                    #display(data[0])
                    lat = data[0]['lat']
                    lon = data[0]['lon']
                    return lat, lon, True
                else:
                    return 0, 0, False
            except IndexError:
                return 0, 0, False
            except TypeError:
                return 0, 0, False
        else:
            return 0, 0, False
    except OSError:
        print("Failed to load {}".format(link))
        return 0, 0, False


In [29]:
print(cities_pretty['wiki article'][0])
get_geo(cities_pretty['wiki article'][0])

https://nominatim.openstreetmap.org/search.php?q=ירושלים&format=json


('31.778345', '35.2250786', True)

In [30]:
lats = []
lons = []
valids = []

import ipywidgets as widgets

bar = widgets.IntProgress(
    value=0,
    min=0,
    max=len(cities_pretty['wiki article']),
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)


display(bar)

for n, i in enumerate(cities_pretty['wiki article']):
    lat, lon, valid_geo = get_geo(i)
    lats.append(lat)
    lons.append(lon)
    valids.append(valid_geo)
    bar.value = (n+1)




IntProgress(value=0, description='Loading:', max=153)

In [39]:
cities_pretty['lat'] = lats
cities_pretty['lon'] = lons
cities_pretty['valid GEO'] = valids
cities_pretty['text'] = 'At:\n' + cities_pretty['city names'] +'\nSick: '+ cities_pretty['sick'].astype(str) + '\nPopulation: ' + cities_pretty['population'].astype(str) 

In [40]:
display(HTML(cities_pretty.sort_values(by=['population'],ascending=False).head().to_html(escape=False)))
cities_pretty.sort_values(by=['population'],ascending=False).to_csv('CompleteCities.csv')

,population,sick,city names,wiki article,wiki,lat,lon,valid GEO,text,percent
0,914559,1132,ירושלים,https://nominatim.openstreetmap.org/search.php?q=ירושלים&format=json,link,31.778345,35.2250786,True,At:\nירושלים\nSick: 1132\nPopulation: 914559,0.001238
2,450192,337,תל אביב - יפו,https://nominatim.openstreetmap.org/search.php?q=תל אביב - יפו&format=json,link,32.0804808,34.7805274,True,At:\nתל אביב - יפו\nSick: 337\nPopulation: 450192,0.000749
14,272584,96,חיפה,https://nominatim.openstreetmap.org/search.php?q=חיפה&format=json,link,32.8191218,34.9983856,True,At:\nחיפה\nSick: 96\nPopulation: 272584,0.000352
4,242478,138,פתח תקווה,https://nominatim.openstreetmap.org/search.php?q=פתח תקווה&format=json,link,32.0877639,34.8859985,True,At:\nפתח תקווה\nSick: 138\nPopulation: 242478,0.000569
5,241010,132,ראשון לציון,https://nominatim.openstreetmap.org/search.php?q=ראשון לציון&format=json,link,31.9635712,34.8101149,True,At:\nראשון לציון\nSick: 132\nPopulation: 241010,0.000548


In [41]:
import plotly.express as px
import plotly.io as pio
df = cities_pretty[cities_pretty['valid GEO']==True]
fig = px.scatter_geo(df,
                     locationmode = 'country names',
                    lon = df['lon'],
                    lat = df['lat'],
                     hover_name="text", # column added to hover information
                     size="sick", # size of markers
                     projection = 'natural earth',
                    #projection_scale= 0.1,
                    center = {'lat': 31.778345, 'lon':35.2250786})
fig.update_geos(
    showcountries=True, countrycolor="RebeccaPurple")

fig.show()
pio.write_html(fig,'index.html',auto_open=True)

In [38]:
cities_pretty['percent'] = cities_pretty['sick']/cities_pretty['population']
display(HTML(cities_pretty.sort_values(by=['percent'],ascending=False).head().to_html(escape=False)))


,population,sick,city names,wiki article,wiki,lat,lon,valid GEO,text,percent
32,5713,36,קרית יערים,https://nominatim.openstreetmap.org/search.php?q=קרית יערים&format=json,link,31.8035128,35.1012545,True,Ammount of sick at: קרית יערים 36 population at city 5713,0.006301
29,6724,39,"כפר חב""ד","https://nominatim.openstreetmap.org/search.php?q=כפר חב""ד&format=json",link,31.9907676,34.849123,True,"Ammount of sick at: כפר חב""ד 39 population at city 6724",0.005800
24,10676,61,אפרת,https://nominatim.openstreetmap.org/search.php?q=אפרת&format=json,link,31.6536462,35.1508714,True,Ammount of sick at: אפרת 61 population at city 10676,0.005714
1,195298,1061,בני ברק,https://nominatim.openstreetmap.org/search.php?q=בני ברק&format=json,link,32.0873899,34.8324376,True,Ammount of sick at: בני ברק 1061 population at city 195298,0.005433
35,8602,32,כוכב יעקב,https://nominatim.openstreetmap.org/search.php?q=כוכב יעקב&format=json,link,31.8814286,35.2452887,True,Ammount of sick at: כוכב יעקב 32 population at city 8602,0.003720
